In [1]:
import sys
from tqdm import tqdm

# TO CHANGE
BASEDIR = "../.."
sys.path.insert(0, BASEDIR)

from src.utils import Logger, ReturnStatus, ReturnInfo, AgentTaskSolver, AgentTaskSolverConfig
from src.agents import AgentDriver, AgentDriverConfig
from src.agents.configs import DEFAULT_AGENT_CONFIGS
from src.pipelines.qa.kg_reasoning.weak_reasoner.query_parser.configs import DEFAULT_KW_EXTRACTION_TASK_CONFIG
from src.pipelines.qa.kg_reasoning.weak_reasoner.answer_generator.configs import DEFAULT_ANSWER_GEN_TASK_CONFIG
from src.pipelines.memorize.extractor.configs import DEFAULT_EXTRACT_TRIPLETS_TASK_CONFIG, DEFAULT_EXTRACT_THESISES_TASK_CONFIG

/home/dzigen/Desktop/PersonalAI/.pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
agent_conifg = AgentDriverConfig(
    name = 'ollama', agent_config = DEFAULT_AGENT_CONFIGS['ollama'])

In [3]:
agent_conifg

AgentDriverConfig(name='ollama', agent_config=AgentConnectorConfig(gen_strategy={'num_predict': 2048, 'seed': 42, 'top_k': 1, 'temperature': 0.0}, credentials={'host': 'http://localhost:11434'}, ext_params={'model': 'llama3.2', 'timeout': 560, 'keep_alive': -1}))

In [4]:
agent = AgentDriver.connect(agent_conifg)

In [5]:
kw_extraction_solver = AgentTaskSolver(agent, DEFAULT_KW_EXTRACTION_TASK_CONFIG)
ag_solver = AgentTaskSolver(agent, DEFAULT_ANSWER_GEN_TASK_CONFIG)

In [6]:
output = kw_extraction_solver.solve(lang='en', query="Which people have positive opinion about display of Mi 10pro on 22.12.2018?")

In [10]:
output = ag_solver.solve(lang='en', triplets=[], query="The majority of speakers have positive, neutral or negative sentiment about connection of Apple?")

In [7]:
output

(['people', 'opinion', 'display', 'Mi 10pro', '22.12.2018'],
 <ReturnStatus.success: 0>)

In [6]:
import json

with open("/home/dzigen/Desktop/PersonalAI/Personal-AI/data/qa_datasets/diaasqa/Augment_DiaASQ.json", 'r', encoding='utf-8') as fd:
    data = json.loads(fd.read())

In [7]:
item_idx = 2

date_txt = data['data'][item_idx]['time'].split(',')[0]
dialogue_txt = data['data'][item_idx]['text_dialog']

expanded_dialogue = f"Date of the dialogue: {date_txt}\n{dialogue_txt}"

In [8]:
print(expanded_dialogue)

Date of the dialogue: 25.11.2020
Kayla: "My brother 's GT2PRO is not as hot as his IQOO7 calling for a voice call . Xiaomi 10Pro 's video is not hot . I suddenly felt that 10PRO would not get stuck . I hope 9000 will be better ."
Gerld: "10PRO video is not hot ? Is there a mobile phone in this world that does not get hot when using WeChat video ?"
Kayla: "My family is poor , and the mobile phone is not hot when playing games at cold night [ cute ]"
Gerld: "Playing games is not necessarily hotter than playing videos [ black line][black line][black line ]"
Herbert: "What the hell ! ! ! Feel the same ! The chicken is hot ! I return it just now ! Really the first lesson for young people !"
Madison: "10Pro 's back cover is broken , still holding on , really strong"



In [9]:
triplet_extraction_solver = AgentTaskSolver(agent, DEFAULT_EXTRACT_TRIPLETS_TASK_CONFIG)
thesis_extraction_solver = AgentTaskSolver(agent, DEFAULT_EXTRACT_THESISES_TASK_CONFIG)

In [10]:
output = thesis_extraction_solver.solve(text=expanded_dialogue)

In [9]:
output = triplet_extraction_solver.solve(text=expanded_dialogue)

In [22]:
output[0][0].stringified.strip('"\'\n')

'kayla has phone gt2pro'

In [11]:
[item.stringified for item in output[0]]

["kayla's brother has gt2pro",
 "kayla's brother has gt2pro",
 'kayla thinks gt2pro is not as hot as iqoo7',
 'kayla thinks gt2pro is not as hot as iqoo7',
 'xiaomi 10pro video is not hot',
 'xiaomi 10pro video is not hot',
 'xiaomi 10pro video is not hot',
 'kayla hopes 9000 will be better',
 'kayla hopes 9000 will be better',
 'kayla hopes 9000 will be better',
 'gerld thinks 10pro video is not hot',
 'gerld thinks 10pro video is not hot',
 'gerld thinks 10pro video is not hot',
 'gerld thinks 10pro video is not hot',
 'gerld asks if there is a mobile phone that does not get hot when using wechat video',
 'gerld asks if there is a mobile phone that does not get hot when using wechat video',
 'gerld asks if there is a mobile phone that does not get hot when using wechat video',
 "kayla's family is poor",
 "kayla's family is poor",
 'kayla thinks the mobile phone is not hot when playing games at cold night',
 'kayla thinks the mobile phone is not hot when playing games at cold night',


In [12]:
output

([Triplet(start_node=Node(name='brother', type=<NodeType.object: 'object'>, prop={}, stringified='brother', id='e511e2a4a4b88e45e210c770b932afb7'), relation=Relation(name='hyper', type=<RelationType.hyper: 'hyper'>, prop={}, id='b9a32f29af002bd0fff0b46e883406aa'), end_node=Node(name="kayla's brother has gt2pro", type=<NodeType.hyper: 'hyper'>, prop={}, stringified="kayla's brother has gt2pro", id='b9a32f29af002bd0fff0b46e883406aa'), stringified="kayla's brother has gt2pro", id='d46e9a14ee1da778e4839f157c97b851'),
  Triplet(start_node=Node(name='gt2pro', type=<NodeType.object: 'object'>, prop={}, stringified='gt2pro', id='67462a69c2a610e0f591b321dc91c542'), relation=Relation(name='hyper', type=<RelationType.hyper: 'hyper'>, prop={}, id='b9a32f29af002bd0fff0b46e883406aa'), end_node=Node(name="kayla's brother has gt2pro", type=<NodeType.hyper: 'hyper'>, prop={}, stringified="kayla's brother has gt2pro", id='b9a32f29af002bd0fff0b46e883406aa'), stringified="kayla's brother has gt2pro", id='